In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os import path
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import sklearn.preprocessing
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
import src.lib.utility_classfier as uclf
import src.lib.optimal_threhold_related as thres
import src.lib.fairness_tests as fair

In [2]:
data_path='/Users/lifuchen/Desktop/research/data.csv'
df = pd.read_csv(data_path)

In [3]:
y = df.Class.values
X = df.drop(['GRID','Class'], axis=1)
X.shape

(109490, 87)

In [4]:
def get_result (classifier, records, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female):
        method_to_call = getattr(uclf, classifier)
        y_val_score = method_to_call(X_train_scaled, y_train,X_val_scaled, y_val)
        y_test_score = method_to_call(X_train_scaled, y_train,X_test_scaled, y_test)
        
        threshold, ba_val, ba_test = balance_accuracy (y_val, y_val_score,y_test, y_test_score)
        auroc = roc_auc_score(y_val, y_val_score)

        y_val_score_male = method_to_call(X_train_scaled, y_train, X_val_male_scaled, y_val_male)
        y_test_score_male = method_to_call(X_train_scaled, y_train,X_test_male_scaled, y_test_male)
        threshold_male, ba_val_male, ba_test_male = balance_accuracy (y_val_male, y_val_score_male,y_test_male, y_test_score_male)

        y_val_score_female = method_to_call(X_train_scaled, y_train, X_val_female_scaled, y_val_female)
        y_test_score_female = method_to_call(X_train_scaled, y_train,X_test_female_scaled, y_test_female)
        threshold_female, ba_val_female, ba_test_female = balance_accuracy (y_val_female, y_val_score_female, y_test_female, y_test_score_female)

        eod = fair.get_EOD(y_test_male, y_test_score_male,threshold_male, y_test_female, y_test_score_female, threshold_female)
        sp = fair.get_SP(y_test_male, y_test_score_male,threshold_male, y_test_female, y_test_score_female, threshold_female)

        records.append({
            'auroc': auroc,
            'overall threshold': threshold,
            'overall ba validation': ba_val,
            'overall ba test': ba_test,
            'male threshold': threshold_male,
            'male ba validation': ba_val_male,
            'male ba test': ba_test_male,
            'female threshold': threshold_female,
            'female ba validation': ba_val_female,
            'female ba test': ba_test_female,
            'eod': eod,
            'di': sp,
        })

In [5]:
def balance_accuracy (y_val, y_val_score,y_test, y_test_score):
    
    threshold, _ = thres.get_optimal_threshold_Jvalue (y_val, y_val_score)
    print ("Optimal threshold by J value is ",threshold)

    ba_val = thres.calculate_balanced_accuracy(y_val, y_val_score, threshold)
    print ("Balanced accuracy score of val is ", ba_val)

    ba_test = thres.calculate_balanced_accuracy(y_test, y_test_score, threshold)
    print ("Balanced accuracy score of test is ",ba_test)

    return threshold, ba_val, ba_test

In [6]:
def fairness_metrics (X, y, attribute, random_state):
    X_train, y_train, X_val, y_val, X_test, y_test, X_val_female, X_val_male, y_val_female, y_val_male, X_test_female, X_test_male, y_test_female, y_test_male \
        = fair.split_by_trait(X, y, attribute, random_state)
    print("X train", X_train.shape[0])
    print("Y train", y_train.shape[0])
    print(X_val.shape[0], X_val_male.shape[0], X_val_female.shape[0])
    print(y_val.shape[0], y_val_male.shape[0], y_val_female.shape[0])
    print(X_test.shape[0], X_test_male.shape[0], X_test_female.shape[0])
    print(y_test.shape[0], y_test_male.shape[0], y_test_female.shape[0])

    max_abs_scaler = preprocessing.MaxAbsScaler()
    X_train_scaled = max_abs_scaler.fit_transform(X_train)
    X_test_scaled = max_abs_scaler.transform(X_test)
    X_test_male_scaled = max_abs_scaler.transform(X_test_male)
    X_test_female_scaled = max_abs_scaler.transform(X_test_female)
    X_val_scaled = max_abs_scaler.transform(X_val)
    X_val_male_scaled = max_abs_scaler.transform(X_val_male)
    X_val_female_scaled = max_abs_scaler.transform(X_val_female)

    get_result ("logic_regression", records_lr, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)
    get_result ("random_forest", records_rf, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)
    get_result ("decision_tree", records_dt, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)
    get_result ("gradiant_boosting", records_gbt, X_train_scaled, y_train, X_val_scaled, y_val, X_test_scaled, y_test, X_val_male_scaled, y_val_male, X_test_male_scaled, y_test_male, X_val_female_scaled, y_val_female, X_test_female_scaled, y_test_female)

In [7]:
records_lr = []
records_rf = []
records_dt = []
records_gbt = []

fairness_metrics (X, y, "GENDER", 0)

X train 65694
Y train 65694
21898 7782 14116
21898 7782 14116
21898 7707 14191
21898 7707 14191


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2590532945377533
0.26233534084451454
Classification report
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     19904
           1       0.46      0.04      0.07      1994

    accuracy                           0.91     21898
   macro avg       0.69      0.52      0.51     21898
weighted avg       0.87      0.91      0.87     21898

Confusion_matrix
[[19822    82]
 [ 1924    70]]
done in 0.584100s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2590532945377533
0.26252435164193133
Classification report
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     19934
           1       0.42      0.04      0.07      1964

    accuracy                           0.91     21898
   macro avg       0.67      0.52      0.51     21898
weighted avg       0.87      0.91      0.87     21898

Confusion_matrix
[[19839    95]
 [ 1894    70]]
done in 0.528572s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.40900000000000003
threshold:0.2, J-value:0.26299999999999996
threshold:0.30000000000000004, J-value:0.151
threshold:0.4, J-value:0.07200000000000001
threshold:0.5, J-value:0.031000000000000003
threshold:0.6000000000000001, J-value:0.012
threshold:0.7000000000000001, J-value:0.001
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7043300318963321
Balanced accuracy score of test is  0.6973388965664085


...

0.2590532945377533
0.23832982946273593
Classification report
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     12989
           1       0.46      0.04      0.07      1127

    accuracy                           0.92     14116
   macro avg       0.69      0.52      0.51     14116
weighted avg       0.89      0.92      0.89     14116

Confusion_matrix
[[12941    48]
 [ 1086    41]]
done in 0.711277s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2590532945377533
0.2413626826357662
Classification report
              precision    recall  f1-score   support

           0       0.92      0.99      0.96     13074
           1       0.38      0.04      0.07      1117

    accuracy                           0.92     14191
   macro avg       0.65      0.52      0.51     14191
weighted avg       0.88      0.92      0.89     14191

Confusion_matrix
[[13007    67]
 [ 1076    41]]
done in 0.688779s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.41900000000000004
threshold:0.2, J-value:0.267
threshold:0.30000000000000004, J-value:0.162
threshold:0.4, J-value:0.081
threshold:0.5, J-value:0.032
threshold:0.6000000000000001, J-value:0.013000000000000001
threshold:0.7000000000000001, J-value:0.001
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7093304258609924
Balanced accuracy score of test is  0.6939289799856995
True positive rate of class 1 is  0.702
Tr

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.26448598669059525
0.2520034366722554
Classification report
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     13074
           1       0.00      0.00      0.00      1117

    accuracy                           0.92     14191
   macro avg       0.46      0.50      0.48     14191
weighted avg       0.85      0.92      0.88     14191

Confusion_matrix
[[13073     1]
 [ 1117     0]]
done in 0.534844s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.39499999999999996
threshold:0.2, J-value:0.22599999999999998
threshold:0.30000000000000004, J-value:0.10200000000000001
threshold:0.4, J-value:0.019
threshold:0.5, J-value:0.0
threshold:0.6000000000000001, J-value:0.0
threshold:0.7000000000000001, J-value:0.0
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.6972239427491818
Balanced accuracy score of test is  0.6751617300268193
True positive rate of class 1 is  

In [8]:
records_lr = []
records_rf = []
records_dt = []
records_gbt = []
for random_state in range(0,16):
    fairness_metrics (X, y, "GENDER", random_state)

result_lr = pd.DataFrame(records_lr)
result_rf = pd.DataFrame(records_rf)
result_dt = pd.DataFrame(records_dt)
result_gbt = pd.DataFrame(records_gbt)

X train 65694
Y train 65694
21898 7782 14116
21898 7782 14116
21898 7707 14191
21898 7707 14191


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2590532945377533
0.26233534084451454
Classification report
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     19904
           1       0.46      0.04      0.07      1994

    accuracy                           0.91     21898
   macro avg       0.69      0.52      0.51     21898
weighted avg       0.87      0.91      0.87     21898

Confusion_matrix
[[19822    82]
 [ 1924    70]]
done in 0.496086s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2590532945377533
0.26252435164193133
Classification report
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     19934
           1       0.42      0.04      0.07      1964

    accuracy                           0.91     21898
   macro avg       0.67      0.52      0.51     21898
weighted avg       0.87      0.91      0.87     21898

Confusion_matrix
[[19839    95]
 [ 1894    70]]
done in 0.506616s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.40900000000000003
threshold:0.2, J-value:0.26299999999999996
threshold:0.30000000000000004, J-value:0.151
threshold:0.4, J-value:0.07200000000000001
threshold:0.5, J-value:0.031000000000000003
threshold:0.6000000000000001, J-value:0.012
threshold:0.7000000000000001, J-value:0.001
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.7043300318963321
Balanced accuracy score of test is  0.6973388965664085


...

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.25939762162427976
0.24034908426739293
Classification report
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     13047
           1       0.42      0.03      0.06      1122

    accuracy                           0.92     14169
   macro avg       0.67      0.51      0.51     14169
weighted avg       0.88      0.92      0.89     14169

Confusion_matrix
[[12994    53]
 [ 1084    38]]
done in 0.518789s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.25939762162427976
0.23422715871383584
Classification report
              precision    recall  f1-score   support

           0       0.93      1.00      0.96     12908
           1       0.45      0.05      0.08      1066

    accuracy                           0.92     13974
   macro avg       0.69      0.52      0.52     13974
weighted avg       0.89      0.92      0.89     13974

Confusion_matrix
[[12847    61]
 [ 1016    50]]
done in 0.515409s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.39
threshold:0.2, J-value:0.259
threshold:0.30000000000000004, J-value:0.14
threshold:0.4, J-value:0.078
threshold:0.5, J-value:0.030000000000000002
threshold:0.6000000000000001, J-value:0.009000000000000001
threshold:0.7000000000000001, J-value:0.001
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.6951673894750734
Balanced accuracy score of test is  0.6962363465855345
True positive rate of class 1 is  0.69
Tr

0.2652384736672351
0.24421575506547671
Classification report
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     12908
           1       0.12      0.00      0.00      1066

    accuracy                           0.92     13974
   macro avg       0.52      0.50      0.48     13974
weighted avg       0.86      0.92      0.89     13974

Confusion_matrix
[[12901     7]
 [ 1065     1]]
done in 0.525441s
threshold:0.0, J-value:0.0
threshold:0.1, J-value:0.36999999999999994
threshold:0.2, J-value:0.22499999999999998
threshold:0.30000000000000004, J-value:0.078
threshold:0.4, J-value:-0.001
threshold:0.5, J-value:-0.001
threshold:0.6000000000000001, J-value:0.0
threshold:0.7000000000000001, J-value:0.0
threshold:0.8, J-value:0.0
threshold:0.9, J-value:0.0
Optimal threshold by J value is  0.1
Balanced accuracy score of val is  0.6850285004154048
Balanced accuracy score of test is  0.6724378208955745
True positive rate of class 1 is  0.703
True

In [ ]:
def add_mean_sd(records, result_table):
    records.append({
        'auroc': result_table["auroc"].mean(),
        'overall threshold': result_table["threshold"].mean(),
        'male threshold': result_table["threshold_male"].mean(),
        'female threshold': result_table["threshold_female"].mean(),
        'overall ba validation': result_table["ba_val"].mean(),
        'overall ba test': result_table["ba_test"].mean(),
        'male ba validation': result_table["ba_val_male"].mean(),
        'male ba test': result_table["ba_test_male"].mean(),
        'female ba validation': result_table["ba_val_female"].mean(),
        'female ba test': result_table["ba_test_female"].mean(),
        'overall precision':result_table["precision"].mean(),
        'overall recall':result_table["recall"].mean(),
        'overall tpr':result_table["tpr"].mean(),
        'overall tnr':result_table["tnr"].mean(),
        'overall pd':result_table["pd"].mean(),
        'male precision':result_table["precision_male"].mean(),
        'male recall':result_table["recall_male"].mean(),
        'male tpr':result_table["tpr_male"].mean(),
        'male tnr':result_table["tnr_male"].mean(),
        'male pd':result_table["pd_male"].mean(),
        'female precision':result_table["precision_female"].mean(),
        'female recall':result_table["recall_female"].mean(),
        'female tpr':result_table["tpr_female"].mean(),
        'female tnr':result_table["tnr_female"].mean(),
        'female pd':result_table["pd_female"].mean(),
        'eod': result_table["eod"].mean(),
        'di': result_table["sp"].mean(),
        })
    records.append({
        'auroc': result_table["auroc"].std(),
        'overall threshold': result_table["threshold"].std(),
        'male threshold': result_table["threshold_male"].std(),
        'female threshold': result_table["threshold_female"].std(),
        'male ba validation': result_table["ba_val_male"].std(),
        'male ba test': result_table["ba_test_male"].std(),
        'female ba validation': result_table["ba_val_female"].std(),
        'female ba test': result_table["ba_test_female"].std(),
        'overall ba validation': result_table["ba_val"].std(),
        'overall ba test': result_table["ba_test"].std(),
        'overall precision':result_table["precision"].std(),
        'overall recall':result_table["recall"].std(),
        'overall tpr':result_table["tpr"].std(),
        'overall tnr':result_table["tnr"].std(),
        'overall pd':result_table["pd"].std(),
        'male precision':result_table["precision_male"].std(),
        'male recall':result_table["recall_male"].std(),
        'male tpr':result_table["tpr_male"].std(),
        'male tnr':result_table["tnr_male"].std(),
        'male pd':result_table["pd_male"].std(),
        'female precision':result_table["precision_female"].std(),
        'female recall':result_table["recall_female"].std(),
        'female tpr':result_table["tpr_female"].std(),
        'female tnr':result_table["tnr_female"].std(),
        'female pd':result_table["pd_female"].std(),
        'eod': result_table["eod"].std(),
        'di': result_table["sp"].std(),
        })
    records.append({
        'auroc': result_table["auroc"].mean(),
        'overall threshold': result_table["threshold"].mean(),
        'male threshold': result_table["threshold_male"].mean(),
        'female threshold': result_table["threshold_female"].mean(),
        'overall ba test': result_table["ba_test"].mean(),
        'male ba test': result_table["ba_test_male"].mean(),
        'female ba test': result_table["ba_test_female"].mean(),
        'overall tpr':result_table["tpr"].mean(),
        'overall pd':result_table["pd"].mean(),
        'male tpr':result_table["tpr_male"].mean(),
        'male pd':result_table["pd_male"].mean(),
        'female tpr':result_table["tpr_female"].mean(),
        'female pd':result_table["pd_female"].mean(),
        'eod': result_table["eod"].mean(),
        'di': result_table["sp"].mean(),
        })
    pd_result = pd.DataFrame(records)
    return pd_result


In [10]:
result_lr = add_mean_sd (records_lr, result_lr)
result_rf = add_mean_sd (records_rf, result_rf)
result_dt = add_mean_sd (records_dt, result_dt)
result_gbt = add_mean_sd (records_gbt, result_gbt)

result_path='/Users/lifuchen/Desktop/research/results/'
result_lr.to_csv(path.join(result_path,'gender-lr-result.csv'), index=False)
result_rf.to_csv(path.join(result_path,'gender-rf-result.csv'), index=False)
result_dt.to_csv(path.join(result_path,'gender-dt-result.csv'), index=False)
result_gbt.to_csv(path.join(result_path,'gender-gbt-result.csv'), index=False)

In [11]:
def print_result(result_table):
    print ('overall ba validation: %.4f (+/- %.4f)' % (result_table["overall ba validation"].mean(), result_table["overall ba validation"].std()))
    print ('overall ba test: %.4f (+/- %.4f)' % (result_table["overall ba test"].mean(), result_table["overall ba test"].std()))
    print ('male ba validation: %.4f (+/- %.4f)' % (result_table["male ba validation"].mean(), result_table["male ba validation"].std()))
    print ('male ba test: %.4f (+/- %.4f)' % (result_table["male ba test"].mean(), result_table["male ba test"].std()))
    print ('female ba validation: %.4f (+/- %.4f)' % (result_table["female ba validation"].mean(), result_table["female ba validation"].std()))
    print ('female ba test: %.4f (+/- %.4f)' % (result_table["female ba test"].mean(), result_table["female ba test"].std()))
    print ('eod: %.4f (+/- %.4f)' % (result_table["eod"].mean(), result_table["eod"].std()))
    print ('di: %.4f (+/- %.4f)' % (result_table["di"].mean(), result_table["di"].std()))
